In [1]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from bson import ObjectId
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

In [2]:
ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = './ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [9]:
conn.rollback()
from datetime import datetime

# 기관 리스트 정의
institutions = [
    "First City Monument Bank Ltd Lagos",
    "Polaris Bank Limited Lagos",
    "Arve Limited",
    "Access Bank Plc Lagos",
    "Guaranty Trust Bank Plc",
    "BetterLending Company Limited",
    "Credit Direct Ltd Lagos",
    "Zedvance Limited Lagos",
    "NewEdge Finance",
    "MIM Finance Company Limited",
    "MYCREDIT INVESTMENTS LIMITED",
    "Fairmoney Microfinance Bank Limited",
    "Carbon Microfinance Bank Limited",
    "Aella Financial Solution Ltd",
    "ALTRACRED FINANCE INVESTMENT LIMITED",
    "CashXpress Nigeria Limited",
    "RenMoney MFB Nigeria Limited Lagos",
    "BLUERIDGE MICROFINANCE BANK LIMITED",
    "BRANCH INTERNATIONAL FINANCIAL SERVICE LIMITED"
]

# 결과를 저장할 리스트
all_results = []

# 각 기관별로 연도별 부실률 계산
for institution in institutions:
    result_row = {'전체기관': institution}
    
    # 각 연도별 부실률 계산
    for year in range(2015, 2025):
        qry = """
            SELECT 
                COUNT(ch."createdAt"),
                SUM(CASE WHEN ch."status" = 'lost' THEN 1 ELSE 0 END)
            FROM credit_history ch
            WHERE ch."openedAt" >= %s 
            AND ch."openedAt" < %s
            AND ch."institutionName" = %s
        """
        
        start_date = f'{year}-01-01'
        end_date = f'{year+1}-01-01'
        
        cursor.execute(qry, (start_date, end_date, institution))
        result = cursor.fetchone()
        
        # None인 경우 0으로 처리
        result_row[f'{year} 대출 건수'] = result[0] if result[0] is not None else 0
        result_row[f'{year} 부실 건수'] = result[1] if result[1] is not None else 0
    
    all_results.append(result_row)

# DataFrame 생성
columns = ['전체기관']
for year in range(2015, 2025):
    columns.extend([f'{year} 대출 건수', f'{year} 부실 건수'])
df = pd.DataFrame(all_results, columns=columns)

# CSV 저장
df.to_csv('./연도별_기관별_대출 및 부실건수.csv', index=False, encoding='utf-8-sig')

# 결과 확인
print(df.head())

                                 전체기관  2015 대출 건수  2015 부실 건수  2016 대출 건수  \
0  First City Monument Bank Ltd Lagos           0           0           2   
1          Polaris Bank Limited Lagos           0           0           0   
2                        Arve Limited           0           0           0   
3               Access Bank Plc Lagos           0           0           1   
4             Guaranty Trust Bank Plc           0           0           0   

   2016 부실 건수  2017 대출 건수  2017 부실 건수  2018 대출 건수  2018 부실 건수  2019 대출 건수  \
0           0           0           0           0           0           0   
1           0           0           0           0           0           0   
2           0           0           0           0           0           0   
3           0           1           0           2           0          34   
4           0           0           0           1           0           9   

   ...  2020 대출 건수  2020 부실 건수  2021 대출 건수  2021 부실 건수  2022 대출 건수  \
0  .